# Final Project

In [1]:
# Import all required libraries
from __future__ import division # For python 2.*

import numpy as np
import matplotlib.pyplot as plt
import mltools as ml
from sklearn import *
from sklearn.metrics import roc_curve, auc

np.random.seed(0)
%matplotlib inline
# Data Loading
X = np.genfromtxt('uci-cs178-f21/X_train.txt', delimiter=',')
Y = np.genfromtxt('uci-cs178-f21/Y_train.txt', delimiter=None)

# The test data
Xte = np.genfromtxt('uci-cs178-f21/X_test.txt', delimiter=',')

print("Shape of X: {}, Xte: {}, Y: {}".format(X.shape, Xte.shape, Y.shape))

Xtr,Xva,Ytr,Yva = ml.splitData(X,Y,0.5)

Shape of X: (7423, 107), Xte: (7422, 107), Y: (7423,)


## kNN

In [2]:
K = [1, 5, 10, 20, 50, 100, 200, 300, 500]
for k in K:
    knn = neighbors.KNeighborsClassifier(n_neighbors=k, n_jobs=-1)
    knn.fit(Xtr, Ytr)
    
    knn_tr_pred = knn.predict_proba(Xtr)
    knn_va_pred = knn.predict_proba(Xva)
    
    fpr_tr, tpr_tr, thresholds = roc_curve(Ytr, knn_tr_pred[:,1])
    knn_tr_auc = metrics.auc(fpr_tr, tpr_tr)
    
    fpr_va, tpr_va, thresholds = roc_curve(Yva, knn_va_pred[:,1])
    knn_va_auc = metrics.auc(fpr_va, tpr_va)
    
    print ("K =", str(k))
    print ("Training AUC: ", str(knn_tr_auc))
    print ("Validation AUC: ", str(knn_va_auc))

K = 1
Training AUC:  1.0
Validation AUC:  0.522992028539224
K = 5
Training AUC:  0.7668977610553699
Validation AUC:  0.5408078366663955
K = 10
Training AUC:  0.6970892558528429
Validation AUC:  0.5526937088208976
K = 20
Training AUC:  0.6608146948160534
Validation AUC:  0.559986404359901
K = 50
Training AUC:  0.6191279960981049
Validation AUC:  0.5740163496287345
K = 100
Training AUC:  0.6054523469435155
Validation AUC:  0.5781037568123453
K = 200
Training AUC:  0.5966256154775177
Validation AUC:  0.5750791626480123
K = 300
Training AUC:  0.5831237225938313
Validation AUC:  0.5687782516239237
K = 500
Training AUC:  0.5691075285674472
Validation AUC:  0.5626514984254622


In [3]:
K = [1, 5, 10, 20, 50, 100, 200, 300, 500]

for k in K:
    knn = ml.knn.knnClassify(Xtr, Ytr, K=k)
    
    print("K = ", str(k))
    print(knn.auc(Xtr, Ytr))
    print(knn.auc(Xva, Yva))

K =  1
1.0
0.522992028539224
K =  5
0.7668977610553698
0.5408078366663955
K =  10
0.6970892558528428
0.5526937088208979
K =  20
0.6608146948160535
0.559986404359901
K =  50
0.6191279960981048
0.5740163496287345
K =  100
0.6054523469435155
0.5781037568123453
K =  200
0.5966256154775176
0.5750791626480124
K =  300
0.5831237225938313
0.5687782516239236
K =  500
0.5691075285674471
0.5626514984254621


In [4]:
# Write into the file

Yhat = knn.predictSoft(Xte)
print(Yhat)
#Yte = np.vstack((np.arange(len(Yhat)), Yhat[:,1])).T
#np.savetxt('f1_knn.txt',Yte,'%d, %.2f',header='Id,Predicted',delimiter=',')

[[0.526 0.474]
 [0.578 0.422]
 [0.402 0.598]
 ...
 [0.42  0.58 ]
 [0.526 0.474]
 [0.496 0.504]]


# Neural Network

In [5]:
X = np.genfromtxt('uci-cs178-f21/X_train.txt', delimiter=',')
Y = np.genfromtxt('uci-cs178-f21/Y_train.txt', delimiter=None)

X,Y = ml.shuffleData(X,Y)
Xtr, Xva, Ytr, Yva = ml.splitData(X, Y, 0.80)

Xte = np.genfromtxt('uci-cs178-f21/X_test.txt', delimiter=',')

Xtr, param = ml.transforms.rescale(Xtr)
Xva, _ = ml.transforms.rescale(Xva,param)
Xte, _ = ml.transforms.rescale(Xte,param)

In [11]:
from sklearn.neural_network import MLPClassifier
nnet = MLPClassifier(solver='lbfgs', hidden_layer_sizes=2, max_iter=256, random_state=1)
nnet.fit(Xtr, Ytr)

/Users/yuhaos/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(hidden_layer_sizes=2, max_iter=256, random_state=1,
              solver='lbfgs')

In [12]:
pred_soft = nnet.predict_proba(Xtr)
false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
roc_auc_tr = auc(false_positive_rate, true_positive_rate)

pred_soft = nnet.predict_proba(Xva)
false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
roc_auc_va = auc(false_positive_rate, true_positive_rate)

print("Training: ", str(roc_auc_tr))
print ("Validation: ", str(roc_auc_va))

Training:  0.7341658908950222
Validation:  0.6876823082951181


In [13]:
layers = [2,3,4,5,6,7,8,9,10]
max = 0
currL = 0
for l in layers:
    nnet = MLPClassifier(solver='lbfgs', hidden_layer_sizes=l, random_state=1)
    nnet.fit(Xtr, Ytr)
    
    pred_soft = nnet.predict_proba(Xtr)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
    roc_auc_tr = auc(false_positive_rate, true_positive_rate)

    pred_soft = nnet.predict_proba(Xva)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
    roc_auc_va = auc(false_positive_rate, true_positive_rate)

    if roc_auc_va > max:
        max = roc_auc_va
        currL = l
    
    print("Layers=", str(l))
    print("Training: ", str(roc_auc_tr))
    print("Validation: ", str(roc_auc_va))

print("Best # Layers: ", currL)
print("AUC: ", max)

/Users/yuhaos/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Layers= 2
Training:  0.720821370565834
Validation:  0.6639504847042433


/Users/yuhaos/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Layers= 3
Training:  0.7109784026308729
Validation:  0.6136306742323097


/Users/yuhaos/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


...

/Users/yuhaos/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Layers= 8
Training:  0.8058730593653933
Validation:  0.6386332138503512


/Users/yuhaos/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Layers= 9
Training:  0.8217902483660575
Validation:  0.6336909357404075
Layers= 10
Training:  0.8226207636769817
Validation:  0.6290915060660591
Best # Layers:  4
AUC:  0.672568859929181


/Users/yuhaos/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
